In [2]:
import random
from names import get_full_name
from tqdm import tqdm
from scipy import stats

class competition:
    
    def __init__(self, nr_players):
        self.player_poule = [player(skill_level = stats.truncnorm((0 - 0.5) / 0.15, (1 - 0.5) / 0.15, loc=0.5, scale=0.15).rvs(1)[0]) for i in tqdm(range(nr_players))]
        
    def play_rand_matches(self, nr_of_games):
        for game in tqdm(range(nr_of_games)):
            players = random.sample(range(0, len(self.player_poule)), 2)
            
            m = match(self.player_poule[players[0]], self.player_poule[players[1]])
            m.simulate_match()
        
    def show_standings(self):
        sorted_poule = sorted(self.player_poule, key=lambda x: x.rating, reverse=True)
        print("leaderboard standings")
        print("------------------------------------")
        for p in sorted_poule[0:10]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), p.rating, p.skill_level))
        for p in sorted_poule[-11:-1]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), p.rating, p.skill_level))
        print("------------------------------------\n")
    

class match:
    
    def __init__(self, player1, player2, k_factor = 32):
        self.p1 = player1
        self.p2 = player2
        self.k_factor = k_factor
        
        self.es_p1 = 1 / (1 + 10**((player2.rating - player1.rating) / 400))
        self.es_p2 = 1 / (1 + 10**((player1.rating - player2.rating) / 400))
    
    def simulate_match(self):
        sl_p1 = self.p1.skill_level
        sl_p2 = self.p2.skill_level
        draw_odds = (1 - abs(sl_p1 - sl_p2)) * (sl_p1 + sl_p2)
        win_chances = [sl_p1, draw_odds, sl_p2]
        
        result_p1 = random.choices([1, 0.5, 0], weights = win_chances, k=1)[0]
        result_p2 = 1-result_p1
        
        self.p1.update_rating(self.k_factor, self.es_p1, result_p1)
        self.p2.update_rating(self.k_factor, self.es_p2, result_p2)
        
        return result_p1

class player:
    
    def __init__(self, rating = 1900, skill_level = 0.5, name = "filler name"):
        self.name = name
        self.rating = rating
        self.skill_level = skill_level
    
    def update_rating(self, k_factor, expected_score, result):
        change = k_factor * (result - expected_score)
        self.rating += round(change)


In [5]:
c = competition(200000)
c.show_standings()
c.play_rand_matches(10000000)
c.show_standings()

100%|██████████| 200000/200000 [01:43<00:00, 1925.97it/s]


leaderboard standings
------------------------------------
filler name           | 1900 | 0.485
filler name           | 1900 | 0.571
filler name           | 1900 | 0.398
filler name           | 1900 | 0.188
filler name           | 1900 | 0.341
filler name           | 1900 | 0.580
filler name           | 1900 | 0.312
filler name           | 1900 | 0.431
filler name           | 1900 | 0.314
filler name           | 1900 | 0.506
filler name           | 1900 | 0.466
filler name           | 1900 | 0.495
filler name           | 1900 | 0.628
filler name           | 1900 | 0.271
filler name           | 1900 | 0.436
filler name           | 1900 | 0.618
filler name           | 1900 | 0.444
filler name           | 1900 | 0.504
filler name           | 1900 | 0.628
filler name           | 1900 | 0.599
------------------------------------



100%|██████████| 10000000/10000000 [01:41<00:00, 98405.45it/s]


leaderboard standings
------------------------------------
filler name           | 2143 | 0.838
filler name           | 2119 | 0.895
filler name           | 2115 | 0.884
filler name           | 2114 | 0.654
filler name           | 2111 | 0.980
filler name           | 2103 | 0.783
filler name           | 2100 | 0.905
filler name           | 2099 | 0.948
filler name           | 2094 | 0.901
filler name           | 2094 | 0.885
filler name           | 1604 | 0.029
filler name           | 1599 | 0.004
filler name           | 1596 | 0.037
filler name           | 1594 | 0.050
filler name           | 1590 | 0.005
filler name           | 1586 | 0.028
filler name           | 1584 | 0.004
filler name           | 1580 | 0.009
filler name           | 1578 | 0.000
filler name           | 1574 | 0.001
------------------------------------



In [10]:
# NOTES
# - gaat er vanuit dat iedereen tegen iedereen speelt
# - match simulatie gaat nog niet goed. de beste moeten meer winnen